## Scripts for scrutinizing the datasets

In [ ]:
#! /usr/bin/env python3

import os
import shutil
import csv
from collections import namedtuple
from datetime import datetime
import urllib.request
import re

import openpyxl


In [ ]:
def list_all_sheets(workbook):
    # non-essential sanity check function
    sheets = [sheet for sheet in available_wb.get_sheet_names()]
    print(sheets)

In [ ]:
def show_all_info(urn):
    # non-essential script that prints all the info related to a urn.
    if urn in main_sheet:
        print("Main example:", main_sheet[urn], "\n")
    if urn in filenames_sheet:
        print("Filenames example:", filenames_sheet[urn], "\n")
    if urn in keywords_sheet:
        print("Keywords example:", keywords_sheet[urn], "\n")
    if urn in advisors_sheet:
        print("Advisors example:", advisors_sheet[urn], "\n")
    if urn in catalog_sheet:
        print("Catalog example:", catalog_sheet[urn], "\n")

In [ ]:
def show_combinations_of_advisors(advisors_sheet):
    urn_advisortitles = dict()
    for urn, advisors_nt_list in advisors_sheet.items():
        for item in advisors_nt_list:
            if item.urn in urn_advisortitles:
                urn_advisortitles[item.urn].append(item.advisor_title)
            else:
                urn_advisortitles[item.urn] = [item.advisor_title, ]

    a_set = set()
    for urn, titles in urn_advisortitles.items():
        for title in titles:
            a_set.add(title)
    print(a_set)

    advisors_permutations = set()

    for urn, titles in urn_advisortitles.items():
        this_permutation = (titles.count('Committee Chair'),
                            titles.count('Committee Co-Chair'),
                            titles.count('Committee Member'),
                            titles.count("Dean's Representative"),
                            )
        advisors_permutations.add(this_permutation)
    for i in advisors_permutations:
        print(i)
    return advisors_permutations

In [ ]:
def find_mismatching_files(filenames_sheet):
    sames = dict()
    for urn, filenames_namedtuple_list in filenames_sheet.items():
        for item in filenames_namedtuple_list:
            if item.urn in sames:
                if sames[item.urn] != item.availability:
                    print('there should be one {}'.format(item.urn))
            else:
                sames[item.urn] = item.availability
    return sames

In [ ]:
def find_misnamed_extensions(filenames_sheet):
    misnamed_urn_filename = []
    for urn, filenames_namedtuple_list in filenames_sheet.items():
        for item in filenames_namedtuple_list:
            if item.filename[-4] != "." and item.filename[-4:] not in ("docx", "r.gz"):
                misnamed_urn_filename.append((urn, item.filename))
                print(urn, item.filename)
    return misnamed_urn_filename

In [ ]:
def find_legacy_school_names(main_sheet):
    schools_etds = dict()
    for urn, itemnamedtuple in main_sheet.items():
        if itemnamedtuple.department in schools_etds:
            schools_etds[itemnamedtuple.department].append(urn)
        else:
            schools_etds[itemnamedtuple.department] = [urn, ]
    for school, urns in schools_etds.items():
        print(school)
    return schools_etds

In [ ]:
def find_page_by_page_pdfs(filenames_sheet):
    split_files = dict()
    for urn, filenames_namedtuples_list in filenames_sheet.items():
        for item in filenames_namedtuples_list:
            if item.urn not in split_files:
                split_files[item.urn] = [item.filename, ]
            else:
                split_files[item.urn].append(item.filename)
    page_by_page_pdfs = []
    for urn, filelist in split_files.items():
        split = False
        for i in filelist:
            if "chap" in i.lower():
                split = True
        if len(filelist) > 1 and split == True:
            print(urn, '\n', filelist, '\n')
            page_by_page_pdfs.append((urn, filelist))
    return page_by_page_pdfs

In [ ]:
def is_catalog_subset_of_database(catalog_sheet, main_sheet):
    outside_uris = []
    for uri in catalog_sheet:
        if uri not in main_sheet:
            # print(uri)
            outside_uris.append(uri)
    print(len(outside_uris))
    return outside_uris

In [ ]:
def are_all_urns_in_main_sheet(main_sheet, catalog_sheet, filenames_sheet, keywords_sheet, advisors_sheet):
    all_urns = make_set_all_urns(main_sheet, catalog_sheet, filenames_sheet, keywords_sheet, advisors_sheet)
    main_urns = set(i for i in main_sheet)
    if (all_urns - main_urns) != set():
        print(all_urns - main_urns)
    else:
        print('all urns in mainsheet')

In [ ]:
# do all files have a pdf, (or no file at all).
def check_for_no_file_objects(main_sheet, catalog_sheet, filenames_sheet, keywords_sheet, advisors_sheet):
    no_files = []
    for urn in make_set_all_urns(main_sheet, catalog_sheet, filenames_sheet, keywords_sheet, advisors_sheet):
        pdf = False
        if urn not in filenames_sheet:
            no_files.append(urn)
            continue
        for nt in filenames_sheet[urn]:
            if 'pdf' in nt.filename.lower():
                pdf = True
        if pdf == False:
            print(urn)
    print('these have no uploaded files:', no_files)

## Reading & parsing source files

In [ ]:
test_items = ('etd-0807101-102716',)

In [ ]:
def read_workbook(workbook_name):
    sourcepath = 'data/databasetables'
    filename = 'prod_etd_{}_database.xlsx'.format(workbook_name)
    fullpath = os.path.join(sourcepath, filename)
    return openpyxl.load_workbook(fullpath)

In [ ]:
def parse_main_sheet(all_db_workbooks):
    """ returns a dictionary in form of:
    {urn: NamedTuple
     urn: NamedTuple
    }
    NamedTuple is expected to have attributes: (urn first_name middle_name last_name suffix author_email
                                                publish_email degree department dtype title abstract availability
                                                availability_description copyright_statement ddate sdate adate
                                                cdate rdate pid url notice notice_response timestamp
                                                survey_completed)
                                            or: (urn first_name middle_name last_name suffix author_email
                                                publish_email degree department dtype title abstract availability
                                                availability_description copyright_statement ddate sdate adate
                                                cdate rdate pid url notices timestamp)
    """
    (available_wb, submitted_wb, withheld_wb) = all_db_workbooks
    main_dict = dict()
    for wb in (available_wb, submitted_wb, withheld_wb):
        current_sheet = wb.get_sheet_by_name('etd_main table')
        for num, row in enumerate(current_sheet.iter_rows()):
            if num == 0:
                headers = (i.value for i in row)
                MainSheet = namedtuple('MainSheet', headers)
                continue
            values = (i.value for i in row)
            item = MainSheet(*values)
            if item.urn not in test_items:
                main_dict[item.urn] = item
    return main_dict

In [ ]:
def parse_filename_sheet(all_db_workbooks):
    """ returns a dictionary in form of:
        urn: [NamedTuple, NamedTuple, ],
        urn: [NamedTuple, ]
    NamedTuple is expected to have attributes (path, size, availability, description, page_count, timestamp)
    """
    (available_wb, submitted_wb, withheld_wb) = all_db_workbooks
    filenames_sheet = dict()
    for wb in (available_wb, submitted_wb, withheld_wb):
        current_sheet = wb.get_sheet_by_name('filename_by_urn table')
        for num, row in enumerate(current_sheet.iter_rows()):
            if num == 0:
                headers = (i.value for i in row)
                Filenames = namedtuple('Filenames', headers)
                continue
            values = (i.value for i in row)
            item = Filenames(*values)
            
            if item.urn not in test_items:
                if item.urn not in filenames_sheet:
                    filenames_sheet[item.urn] = [item, ]
                else:
                    row_timestamp = datetime.strptime(item.timestamp, "%Y-%m-%d %H:%M:%S")
                    filenames_list = [i.filename for i in filenames_sheet[item.urn]]
                    if item.filename in filenames_list:
                        previous_filename_entry = [i for i in filenames_sheet[item.urn] if i.filename == item.filename]
                        previous_timestamp = datetime.strptime(previous_filename_entry[0].timestamp, "%Y-%m-%d %H:%M:%S")
                        if row_timestamp > previous_timestamp:
                            previous_filename_entry[0] = item
                    else:
                        filenames_sheet[item.urn].append(item)
    filenames_sheet = sort_descending_size(filenames_sheet)
    return filenames_sheet

def sort_descending_size(filenames_sheet):
    for urn, list_of_namedtuples in filenames_sheet.items():
        list_of_namedtuples = sorted(list_of_namedtuples, key=lambda x:int(x.size), reverse=True)
    return filenames_sheet
        

In [ ]:
def parse_keyword_sheet(all_db_workbooks):
    """ returns a dictionary in form of:
    {urn: [NamedTuple,
           NamedTuple,
           ]}
    NamedTuple is expected to have attributes ('keyword', 'urn', 'timestamp')
    """
    (available_wb, submitted_wb, withheld_wb) = all_db_workbooks
    keywords_sheet = dict()
    for wb in (available_wb, submitted_wb, withheld_wb):
        current_sheet = wb.get_sheet_by_name('keyword_by_urn table')
        for num, row in enumerate(current_sheet.iter_rows()):
            if num == 0:
                headers = (i.value for i in row)
                Keywords = namedtuple('Keywords', headers)
                continue
            values = (i.value for i in row)
            item = Keywords(*values)
            if item.urn not in test_items:
                if item.urn not in keywords_sheet:
                    keywords_sheet[item.urn] = [item, ]
                else:
                    keywords_sheet[item.urn].append(item)
    return keywords_sheet

In [ ]:
def parse_advisors_sheet(all_db_workbooks):
    """ returns a dictionary in form of:
        {urn: [NamedTuple,
               NamedTuple,
               ]}
        NamedTuple is expected to have attributes ('urn', 'advisor_name', 'advisor_title',
                                                   'advisor_email', 'approval', 'timestamp')
   """
    advisors_sheet = dict()
    (available_wb, submitted_wb, withheld_wb) = all_db_workbooks
    for wb in (available_wb, submitted_wb, withheld_wb):
        current_sheet = wb.get_sheet_by_name('advisor_by_urn table')
        for num, row in enumerate(current_sheet.iter_rows()):
            if num == 0:
                headers = (i.value for i in row)
                Advisors = namedtuple('Advisor', headers)
                continue
            values = (i.value for i in row)
            item = Advisors(*values)
            if item.urn not in test_items:
                if item.urn not in advisors_sheet:
                    advisors_sheet[item.urn] = [item, ]
                else:
                    row_timestamp = datetime.strptime(item.timestamp, "%Y-%m-%d %H:%M:%S")
                    if item.advisor_name in advisors_sheet[item.urn]:
                        previous_advisor_entry = [i for i in advisors_sheet[item.urn] if i.advisor_name == item.advisor_name]
                        previous_timestamp = datetime.strptime(previous_advisor_entry[0].timestamp, "%Y-%m-%d %H:%M:%S")
                        if row_timestamp > previous_timestamp:
                            previous_advisor_entry[0] = item
                    else:
                        advisors_sheet[item.urn].append(item)
    return advisors_sheet


In [ ]:
def parse_catalog_sheet():
    """ returns a dictionary in form of:
        {urn: NamedTuple
         urn: NamedTuple}
    """
    catalog_sheet = dict()
    sourcepath = 'data/Catalogtables'
    sourcefile = 'ETDCatalogRecords20161108.csv'
    with open(os.path.join(sourcepath, sourcefile), encoding='utf-8') as csvfile:
        csvreader = csv.reader(csvfile, delimiter='|')
        for num, row in enumerate(csvreader):
            if num == 0:
                headers = (i for i in row)
                Catalog = namedtuple('Catalog', headers)
                continue
            values = (i for i in row)
            item = Catalog(*values)
            urn = [i for i in os.path.split(item.URL) if 'etd-' in i]
            urn = os.path.split(urn[0])[1]
            if urn not in test_items:
                catalog_sheet[urn] = item
    return catalog_sheet

## Scraping the binaries

In [ ]:
def retrieve_binary(url):
    with urllib.request.urlopen(url) as response:
        return response.read()

In [ ]:
def write_binary_to_file(binary, folder, filename):
    os.makedirs(folder, exist_ok=True)
    filepath = os.path.join(folder, filename)
    with open(filepath, 'bw') as f:
        f.write(binary)

In [ ]:
joined_postupload = ('etd-06182004-122626', 'etd-09012004-114224', 'etd-0327102-091522', 'etd-0707103-142120',
                     'etd-0710102-054039', 'etd-0409103-184148', 'etd-04152004-142117', 'etd-0830102-145811',
                     'etd-0903103-141852', )

In [ ]:
def scrape_binaries(filenames_sheet):
    didnt_grab = []
    target_dir = './ETDbinaries/'
    count = 0
    for num, (urn, filenames_namedtuples_list) in enumerate(filenames_sheet.items()):
        local_dir = os.path.join(target_dir, urn)
        local_files = []
        if os.path.isdir(local_dir):
            local_files = os.listdir(local_dir)
        for item in filenames_namedtuples_list:
            if item.filename in local_files:
                pass
            else:
                url = 'http://etd.lsu.edu/{}/{}'.format("/".join(item.path.split('/')[3:]),
                                                                 item.filename)
                try:
                    binary = retrieve_binary(url)
                    write_binary_to_file(binary, local_dir, item.filename)
                except:
                    count += 1
                    pass
                    didnt_grab.append((urn, item.availability, item.filename))
#                     print(urn, item.availability, item.filename)
    print(count)
    return didnt_grab

In [ ]:
# didnt_grab = scrape_binaries(filenames_sheet)

In [ ]:
def what_files_arent_in_ETD_dump():
    withheld_files = {file for a,b,c in os.walk('/media/francis/ETD/withheld/') for file in c}
    missing_files = set()
    for urn, availability, filename in didnt_grab:
        if filename in withheld_files:
            continue
        else:
    #         print(urn, availability, filename)
            missing_files.add((urn, availability, filename))
    return missing_files

In [ ]:
# missing_files = what_files_arent_in_ETD_dump()

In [ ]:
# print(didnt_grab[:10])

In [ ]:
# missing_list = sorted([*missing_files], key=lambda x:x[0])
# for i in missing_list[:10]:
#     print(i)

In [ ]:
# print(missing_files)

In [ ]:
# ETD_source = [os.path.join(root, file) 
#               for root, dirs, files in os.walk('/media/francis/ETD/')
#               for file in files]

In [ ]:
# for urn, availability, sought_filename in didnt_grab:
#     for file in ETD_source:
#         found_filename = os.path.split(file)[1]
#         found_urn = file.split('/')[5]
#         if sought_filename == found_filename and found_urn == urn:
#             target_file = os.path.join('./ETDbinaries/', urn, found_filename)
#             if not os.path.isfile(target_file):
# #                 print(file, target_file)
#                 shutil.copy2(file, target_file)

## Building the csv

In [ ]:
def csv_writer(data, path):
    with open(path, "w", newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        for line in data:
            writer.writerow(line)

In [ ]:
def concatinate_keywords(keywords_sheet, urn):
    if urn in keywords_sheet:
        return ', '.join(nt.keyword for nt in keywords_sheet[urn] if nt.keyword)
    return ''

In [ ]:
def organize_advisors(advisors_sheet, urn):
    Advisor = namedtuple('Advisor', ('urn', 'advisor_name', 'advisor_title',
                                           'advisor_email', 'approval', 'timestamp'))
    blank_Advisor = Advisor('', '', '', '', '', '')
    Advisors_nt = advisors_sheet[urn]
    advisors_rank = {'Committee Chair': 1, 'Committee Co-Chair': 2, 'Committee Member': 3, "Dean's Representative": 4}
    alpha_Advisors = sorted(Advisors_nt, key=lambda x: x.advisor_name)
    sorted_advisors = sorted(alpha_Advisors, key=lambda x: advisors_rank[x.advisor_title])
    if len(sorted_advisors) > 7 and "Dean's Representative" in sorted_advisors[-1]:
        sorted_advisors = sorted_advisors[:6] + sorted_advisors[-1:]
    elif len(sorted_advisors) > 7:
        sorted_advisors = sorted_advisors[:7]
    elif len(sorted_advisors) < 7:
        missing = 7 - len(sorted_advisors)
        for i in range(missing):
            sorted_advisors.append(blank_Advisor)
    return sorted_advisors

In [ ]:
def strip_slash_and_padding(text):
    if not text:
        return
    text = text.strip()
    if text[-1] == '/':
        text = text[:-1]
    text = text.strip()
    return text

In [ ]:
def replace_null_with_nothing(text):
    if not text:
        return ''
    return text.replace('NULL', '')

In [ ]:
def combine_title(catalog_sheet, main_sheet, urn):
    if urn in catalog_sheet:
        title = catalog_sheet[urn].Title
        title = strip_slash_and_padding(title)
        subtitle = catalog_sheet[urn].Subtitle
        subtitle = strip_slash_and_padding(subtitle)
        if title[-1] == ':':
            title = title[:-1]
        if subtitle:
            csv_title = "{}:  {}".format(title,
                                         subtitle)
        else:
            csv_title = title
    else:
        title = main_sheet[urn].title
        title = strip_slash_and_padding(title)
        csv_title = title
    return csv_title

In [ ]:
def remove_non_nums(text):
    only_nums = ''
    for i in text:
        if i.isnumeric():
            only_nums += i
    return only_nums

In [ ]:
def find_pub_date(catalog_sheet, urn):
    if urn in catalog_sheet:
        if catalog_sheet[urn].SeriesDate:
            return remove_non_nums(catalog_sheet[urn].SeriesDate)
        if catalog_sheet[urn].PubDate:
            return remove_non_nums(catalog_sheet[urn].PubDate)
    return remove_non_nums(main_sheet[urn].ddate)[:4]

In [ ]:
year_month_day = re.compile(r'^(\d{4})[/.-](\d{1,2})[/.-](\d{1,2})$')
year_only = re.compile(r'^(\d{4})$')

In [ ]:
def find_defense_date(main_sheet, urn):
    if main_sheet[urn].ddate:
        return main_sheet[urn].ddate
    return ''

def find_submission_date(main_sheet, urn):
    if main_sheet[urn].sdate:
        return main_sheet[urn].sdate
    return ''

In [ ]:
Filenames = namedtuple('Filenames', ['urn', 'filename', 'path', 'size', 'page_count', 'availability', 'description', 'timestamp'])
special_cases = {'etd-07062015-144158': Filenames(urn='etd-07062015-144158', filename='Moran_Thesis.pdf', path='/export/ETD-db/docs/withheld/etd-07062015-144158/withheld', size='1684241', page_count='NULL', availability='withheld', description='NULL', timestamp='2015-07-07 17:15:34'),
                 'etd-07062015-144155': Filenames(urn='etd-07062015-144155', filename='Morris_Thesis.pdf', path='/export/ETD-db/docs/submitted/etd-07062015-144155/withheld', size='805581', page_count='NULL', availability='withheld', description='NULL', timestamp='2015-07-08 08:59:42'), 
                 'etd-04032015-080642': Filenames(urn='etd-04032015-080642', filename='wang_diss.pdf', path='/export/ETD-db/docs/withheld/etd-04032015-080642/withheld', size='6969877', page_count='NULL', availability='withheld', description='NULL', timestamp='2015-04-13 14:39:16'),
                 'etd-08082016-164729': Filenames(urn='', filename='', path='', size='', page_count='', availability='', description='', timestamp=''),
                 'etd-06062010-192030': Filenames(urn='', filename='', path='', size='', page_count='', availability='', description='', timestamp=''),
                 }

def find_thesis(filenames_sheet, urn):
    if urn in special_cases:
        return special_cases[urn]
    largest_pdf_file = None
    for file in filenames_sheet[urn]:
        extension = file.filename.split('.')[-1].lower().strip()
        if extension != 'pdf':
            continue
        if not largest_pdf_file:
            largest_pdf_file = file
        elif int(file.size) > int(largest_pdf_file.size):
            largest_pdf_file = file
    return largest_pdf_file   

In [ ]:
def parse_dropbox_url(urn):
    thesis_file = find_thesis(filenames_sheet, urn)
    return 'https://dl.dropboxusercontent.com/u/302551934/{}/{}'.format(urn, thesis_file.filename)

def parse_gdrive_to_url():
    gdrive_dict = dict()
    for urn, FilenamesNT in special_cases.items():
        if FilenamesNT.filename == '':
            gdrive_dict[urn] = ''
    with open('data/gdrive_to_url.csv', encoding='utf-8') as csvfile:
        csvreader = csv.reader(csvfile, delimiter='\t')
        for urn, name, url in csvreader:
            if urn in special_cases:
                if name == special_cases[urn].filename:
                    gdrive_dict[urn] = url
                continue
            elif urn not in filenames_sheet:
                gdrive_dict[urn] = ''
                continue
            elif name == find_thesis(filenames_sheet, urn).filename:
                gdrive_dict[urn] = url
    return gdrive_dict


In [ ]:
def make_set_all_urns(main_sheet, catalog_sheet, filenames_sheet, keywords_sheet, advisors_sheet):
    all_urns = set()
    for sheet in (main_sheet, catalog_sheet, filenames_sheet, keywords_sheet, advisors_sheet):
        for urn in sheet:
            all_urns.add(urn)
    all_urns.remove(None)  #
    return all_urns

In [ ]:
overlooked_degrees = set()

def read_legacy_dept_map():
    legacy_current = dict()
    sourcepath = 'data/LegacyNames.csv'
    with open(sourcepath, encoding='utf-8') as csvfile:
        csvreader = csv.reader(csvfile, delimiter='\t')
        for num, row in enumerate(csvreader):
            if num == 0:
                headers = (i for i in row)
            current, legacy = row[0].strip(), row[1].strip()
            if current and current != 'Program no longer active':
                if legacy not in legacy_current:
                    if legacy not in ('New', ):
                        legacy_current[legacy] = current
                else:
                    print(legacy, 'has two mappings')
    return legacy_current

def lookup_current_dept(legacy_dept):
    legacy_current = read_legacy_dept_map()
    if legacy_dept in legacy_current:
        return legacy_current[legacy_dept]
    else:
        overlooked_degrees.add(legacy_dept)
        return legacy_dept

In [ ]:
non_matching_degree_abbrevs = dict()

abbr_degree = {"MPT": "Master of Physical Therapy (MPT)",
    "MUP": "Master of Urban Planning (MUP)",
    "DM": "Doctor of Music (DM)",
    "MTS": "Master of Theological Studies (MTS)",
    "AuD": "Doctor of Audiology (AuD)",
    "MSEE": "Master of Science in Electrical Engineering (MSEE)",
    "MSIB": "Master of Science in International Business (MSIB)",
    "MCSM": "Master of Construction Science and Management (MCSM)",
    "PsyD": "Doctor of Psychology (PsyD)",
    "MSEM": "Master of Science in Engineering Management (MSEM)",
    "MSMSE": "Master of Science in Materials Science and Engineering (MSMSE)",
    "RhD": "Doctor of Rehabilitation (RhD)",
    "MATE": "Master of Arts in the Teaching of English (MATE)",
    "DPT": "Doctor of Physical Therapy (DPT)",
    "MSAgE": "Master of Science in Agricultural Engineering (MSAgE)",
    "PhDOtol": "PhD Otolaryngology (PhDOtol)",
    "MSHRM": "Master of Science in Human Resources Management (MSHRM)",
    "MIM": "Master of International Management (MIM)",
    "DMin": "Doctor of Ministry (DMin)",
    "MSIE": "Master of Science in Industrial Engineering (MSIE)",
    "MSISE": "Master of Science in Infrastructure Systems Engineering (MSISE)",
    "DPA": "Doctor of Public Administration (DPA)",
    "HSOP": "Doctor of Philosophy in Health Services Research (HSOP)",
    "MMatSE": "Master of Materials Science and Engineering (MMatSE)",
    "MAeroE": "Master of Aeronautical Engineering (MAeroE)",
    "MMT": "Master in Management of Technology (MMT)",
    "MSJ": "Master of Science in Jurisprudence (MSJ)",
    "MHP": "Master of Historic Preservation (MHP)",
    "DEng": "Doctor of Engineering (DEng)",
    "MBA": "Master of Business Administration (MBA)",
    "MRED": "Master of Real Estate Development (MRED)",
    "MCTE": "Master of Career and Technology Education (MCTE)",
    "MSAeroE": "Master of Science in Aerospace Engineering (MSAeroE)",
    "MAR": "Master of Arts in Religion (MAR)",
    "MST": "Master's of Science in Teaching (MST)",
    "MJS": "Master of Judicial Studies (MJS)",
    "MALA": "Master of Arts in Liberal Arts (MALA)",
    "MSETM": "Master of Science in Environmental Technology Management (MSETM)",
    "MSHTM": "Master of Science in Hospitality and Tourism Management (MSHTM)",
    "Th.M": "Master of Theology (Th.M)",
    "MSM": "Master of Science in Management (MSM)",
    "MCRP": "Master of City and Regional Planning (MCRP)",
    "MBS": "Master of Building Science (MBS)",
    "MAIS": "Master of Arts in Interdisciplinary Studies (MAIS)",
    "DBA": "Doctor of Business Administration (DBA)",
    "MPH": "Master of Public Health (MPH)",
    "MIDS": "Master of Interdisciplinary Studies (MIDS)",
    "MPA/JD": "Master of Public Administration/Juris Doctorate (MPA/JD)",
    "PhD": "Doctor of Philosophy (PhD)",
    "DMgt": "Doctor of Management (DMgt)",
    "MCIS": "Master of Computer and Information Science (MCIS)",
    "MAE": "Master of Arts in Education (MAE)",
    "MHD": "Master of Human Development (MHD)",
    "MM": "Master of Music (MM)",
    "MGS": "Master of General Studies (MGS)",
    "MSN": "Master of Science in Nursing (MSN)",
    "M.Div": "Master of Divinity (M.Div)",
    "MAC": "Master of Arts in Counseling (MAC)",
    "MCJ": "Master of Criminal Justice (MCJ)",
    "MFR": "Master of Forest Resources (MFR)",
    "MSSS": "Master of Science in Computer Science (MSCS)",
    "MSA": "Master of Science in Administration (MSA)",
    "MURP": "Master of Urban and Regional Planning (MURP)",
    "MAS": "Master in Advanced Studies (MAS)",
    "ND": "Doctor of Nursing (ND)",
    "ME": "Master of Engineering (ME)",
    "MSCRP": "Master of Science in Community and Regional Planning (MSCRP)",
    "MArch": "Master of Architecture (MArch)",
    "MLIS": "Master of Library and Information Science (MLIS)",
    "MSOtol": "MS Otolaryngology (MSOtol)",
    "MLS": "Master of Library Science/Master of Life Sciences (MLS)",
    "MSMANFE": "Master of Science in Manufacturing Engineering (MSMANFE)",
    "MSSE": "Master of Science and Software Engineering (MSSE)",
    "MEngr": "Master of Engineering (MEngr)",
    "MSB": "Masters of Science in Bioscience (MSB)",
    "PED": "Doctor of Physical Education (PED)",
    "MFA": "Master of Fine Arts (MFA)",
    "MMC": "Master of Mass Communication (MMC)",
    "MSBAE": "Master of Science in Biological and Agricultural Engineering (MSBAE)",
    "MAgEd": "Master of Agricultural Education (MAgEd)",
    "MSECE": "Master of Science in Electrical and Computer Engineering (MSECE)",
    "DMD": "Doctor of Dental Medicine (DMD)",
    "MSMatSE": "Master of Science in Material Science Engineering (MSMatSE)",
    "MAPC": "Master of Arts in Pastoral Counseling (MAPC)",
    "MSEd": "Master of Science in Education (MSEd)",
    "DPDS": "Doctor of Planning and Development Studies (DPDS)",
    "MRP": "Master of Regional Planning (MRP)",
    "MNS": "Master of Natural Sciences (MNS)",
    "EdD": "Doctor of Education (EdD)",
    "DrPH": "Doctor of Public Health (DrPH)",
    "DNS": "Doctor of Nursing Science (DNS)",
    "MSIEOR": "Master of Science in Industrial Engineering and Operations Research (MSIEOR)",
    "MAT": "Master of Arts in Teaching (MAT)",
    "MEE": "Master of Electrical Engineering (MEE)",
    "MS": "Master of Science (MS)",
    "MSECO": "Master of Science in Economics (MSECO)",
    "MLA": "Master of Landscape Architecture (MLA)",
    "PhDSurg": "PhD Surgergy (PhDSurg)",
    "MSES": "Master of Science in Engineering Science (MSES)",
    "MHI": "Masters of Health Informatics (MHI)",
    "MSME": "Master of Science in Mechanical Engineering (MSME)",
    "MMUS": "Master of Music (MMUS)",
    "MSW": "Master of Social Work (MSW)",
    "MME": "Master of Music Education (MME)",
    "DMA": "Doctor of Musical Arts (DMA)",
    "MPA": "Master of Public Administration (MPA)",
    "DA": "Doctor of Arts (DA)",
    "MApStat": "Master of Applied Statistics (MApStat)",
    "MSP": "Master of Science in Planning (MSP)",
    "MPP": "Master of Public Policy (MPP)",
    "MSExpSurg": "Medical Surgeon in Experimental Surgery (MSExpSurg)",
    "EdS": "Education Specialist (EdS)",
    "MF": "Master of Forestry (MF)",
    "MPlan": "Master of Planning (MPlan)",
    "MBT": "Master of Business Taxation (MBT)",
    "HSD": "Doctor of Health and Safety (HSD)",
    "MHRD": "Master of Human Resource Development (MHRD)",
    "MSPH": "Master of Science in Public Health (MSPH)",
    "MChE": "Master of Chemical Engineering (MChE)",
    "MSPE": "Master of Science in Petroleum Engineering (MSPE)",
    "MCompE": "Master of Computer Engineering (MCompE)",
    "MT": "Master in Taxation (MT)",
    "MAcc": "Master of Accounting (MAcc)",
    "MPM": "Master of Public Management (MPM)",
    "MSE": "Master of Science in Engineering (MSE)",
    "DME": "Doctor of Music Education (DME)",
    "DSW": "Doctor of Social Work (DSW)",
    "MSCE": "Master of Science in Civil Engineering (MSCE)",
    "DVM": "Doctor of Veterinary Medicine (DVM)",
    "MCE": "Master of Civil Engineering (MCE)",
    "MES": "Master of Environmental Studies (MES)",
    "MECom": "Master of Electronic Commerce (MECom)",
    "MHA": "Master of Health Administration (MHA)",
    "PharmD": "Doctor of Pharmacy (PharmD)",
    "MA": "Master of Arts (MA)",
    "Ded": "Doctor of Education (Ded)",
    "MEnvE": "Master of Environmental Engineering (MEnvE)",
    "ReD": "Doctor of Recreation (ReD)",
    "JD": "Juris Doctorate (JD)",
    "MSBiosyAgE": "Master of Science in Biosystems and Agricultural Engineering (MSBiosyAgE)",
    "PMBA": "Professional Master of Business Administration (PMBA)",
    "MHAMS": "Master of Historical Administration and Museum Studies (MHAMS)",
    "MSIS": "Master of Science in Interdisciplinary Studies (MSIS)",
    "IMES": "International Master of Environmental Sciences (IMES)",
    "MSChE": "Master of Science in Chemical Engineering (MSChE)",
    "MPAcc": "Master of Professional Accounting (MPAcc)",
    "MGIS": "Master of Geographic Information Science (MGIS)",
    "MBioSci": "Master of Biological Science (MBioSci)",
    "MCM": "Master of Construction Management (MCM)",
    "MSMS": "Master of Science in Medical Sciences (MSMS)",
    "MD": "Medical Doctor (MD)",
    "Medical Science": "Doctor of Philosophy (Medical Science)",
    "MGeoE": "Master of Geomechanics Engineering (MGeoE)",
    "MEd": "Master of Education (MEd)",
    "MAM": "Master in Agricultural Management (MAM)",
    "MPRTM": "Master of Parks, Recreation and Tourism Management (MPRTM)",
    "MAgr": "Master of Agriculture (MAgr)",
    }

def expand_degree_type(degree_name):
    if degree_name in abbr_degree:
        return abbr_degree[degree_name]
    else:
        if degree_name not in non_matching_degree_abbrevs:
            non_matching_degree_abbrevs[degree_name] = []
        

In [ ]:
def split_email(text):
    if not text:
        return ''
    list_a = text.split(',')
    list_b = list()
    for i in list_a:
        split_i = i.split(';')
        for n in split_i:
            list_b.append(n.split(' '))
    emails = [item for sublist in list_b for item in sublist]
    
    for i in emails:
        if 'lsu.edu' not in i and len(i) and verify_email(i):
            return i.strip().replace('\\','').replace('/','')
    else:
        fallback_email = emails[0].strip().replace('\\','').replace('/','')
        if verify_email(fallback_email):
            return fallback_email
        else:
            return ''

def verify_email(text):
    if re.match(r"^[^@]+@[^@]+\.[^@]+$", text):
        return True
    return False
    
def lookup_email(urn):
    pack = main_sheet[urn]
    if not pack.author_email:
        return ''
    one_email = split_email(pack.author_email)
    if not one_email:
        return ''
    if verify_email(one_email):
        return one_email
    else:
        one_email = "{}.edu".format(one_email)
        if verify_email(one_email):
            return one_email
        else:
            print('{} email didnt work {}'.format(urn, one_email))
            return ''

In [ ]:
def lookup_hide_email(urn):
    pack = main_sheet[urn]
    translation = {'yes': 'false', 'no': 'true', 'on': 'true'}
    yes_or_no = pack.publish_email.lower()
    return translation[yes_or_no]

In [ ]:
# current_department & department mapped -- we need department & legacy department.

def build_csv(main_sheet, catalog_sheet, filenames_sheet, keywords_sheet, advisors_sheet):
    all_urns = make_set_all_urns(main_sheet, catalog_sheet, filenames_sheet, keywords_sheet, advisors_sheet)
    for papertype in ('dissertation', 'thesis', 'major_paper'):
        csv_data = []
        csvfieldnames = ["title",
                         "fulltext_url",
                         'keywords',
                         'abstract',
                         "author1_fname",
                         'author1_mname',
                         'author1_lname',
                         'author1_suffix',
                         'author1_email',
                         'author1_institution',
                         'advisor1',
                         'advisor2',
                         'advisor3',
                         "advisor1_email",
                         'advisor1_title',
                         "advisor2_email",
                         'advisor2_title',
                         "advisor3_email",
                         'advisor3_title',
                         'advisor4',
                         "advisor4_email",
                         'advisor4_title',
                         'advisor5',
                         "advisor5_email",
                         'advisor5_title',
                         'advisor6',
                         "advisor6_email",
                         'advisor6_title',
                         'advisor7',
                         "advisor7_email",
                         'advisor7_title',
                         'availability',
                         'availability_description',
                         'disciplines',
                         'defense_date',
                         'degree_name',
                         'department',
                         "document_type",
                         'file_name',
                         'file_size',
                         'hide_author_email',
                         'legacy_department',
                         'publication_date',
                         'season',
                         'submission_date',
                         'urn',
                        ]
        csv_data.append(csvfieldnames)

        papertype_urns = [urn for urn in all_urns if main_sheet[urn].dtype.lower() == papertype]
        for urn in papertype_urns:
            csv_title = combine_title(catalog_sheet, main_sheet, urn)
            csv_urn = urn
            fulltext_url = parse_dropbox_url(urn)
#             fulltext_url = gdrive_dict[urn]
            csv_first_name = main_sheet[urn].first_name
            csv_middle_name = main_sheet[urn].middle_name
            csv_last_name = main_sheet[urn].last_name
            csv_suffix = main_sheet[urn].suffix
            csv_suffix = replace_null_with_nothing(csv_suffix)
            csv_author_email = lookup_email(urn)
            csv_hide_author_email = lookup_hide_email(urn)
            sorted_advisors = organize_advisors(advisors_sheet, urn)
            csv_document_type = main_sheet[urn].dtype.lower()
            csv_degree = expand_degree_type(main_sheet[urn].degree)
            csv_legacy_department = main_sheet[urn].department
            csv_department = lookup_current_dept(main_sheet[urn].department)
            csv_disciplines = ''
            csv_keywords = concatinate_keywords(keywords_sheet, urn)
            csv_abstract = main_sheet[urn].abstract
            csv_publication_date = find_pub_date(catalog_sheet, urn)
            csv_defense_date = find_defense_date(main_sheet, urn)
            csv_submission_date = find_submission_date(main_sheet, urn)
            csv_availability = main_sheet[urn].availability
            csv_availability_desc = main_sheet[urn].availability_description
            csv_filename = find_thesis(filenames_sheet, urn).filename
            csv_filesize = find_thesis(filenames_sheet, urn).size
            csv_season = ''

            if urn in filenames_sheet:
                filename = filenames_sheet[urn][0].filename
            else:
                filename = ''
            csv_data.append([csv_title,
                             fulltext_url,
                             csv_keywords,
                             csv_abstract,
                             csv_first_name,
                             csv_middle_name,
                             csv_last_name,
                             csv_suffix,
                             csv_author_email,
                             'Louisiana State University and Agricultural and Mechanical College',
                             sorted_advisors[0].advisor_name,
                             sorted_advisors[1].advisor_name,
                             sorted_advisors[2].advisor_name,
                             sorted_advisors[0].advisor_email,
                             sorted_advisors[0].advisor_title,
                             sorted_advisors[1].advisor_email,
                             sorted_advisors[1].advisor_title,
                             sorted_advisors[2].advisor_email,
                             sorted_advisors[2].advisor_title,
                             sorted_advisors[3].advisor_name,
                             sorted_advisors[3].advisor_email,
                             sorted_advisors[3].advisor_title,
                             sorted_advisors[4].advisor_name,
                             sorted_advisors[4].advisor_email,
                             sorted_advisors[4].advisor_title,
                             sorted_advisors[5].advisor_name,
                             sorted_advisors[5].advisor_email,
                             sorted_advisors[5].advisor_title,
                             sorted_advisors[6].advisor_name,
                             sorted_advisors[6].advisor_email,
                             sorted_advisors[6].advisor_title,
                             csv_availability,
                             csv_availability_desc,
                             csv_disciplines,
                             csv_defense_date,
                             csv_degree, 
                             csv_department,
                             csv_document_type,
                             csv_filename,
                             csv_filesize,
                             csv_hide_author_email,
                             csv_legacy_department,
                             csv_publication_date,
                             csv_season,
                             csv_submission_date,
                             urn,
                             ])
    #     print(csv_data)
        csv_writer(csv_data, '../../scrap{}.csv'.format(papertype))

# if __name__ == '__main__':

In [ ]:
available_wb = read_workbook('available')
submitted_wb = read_workbook('submitted')
withheld_wb = read_workbook('withheld')

In [ ]:
# merges the matching sheets from all 3 workbooks into one datastructure per sheet-type.
all_db_workbooks = (available_wb, submitted_wb, withheld_wb)

main_sheet = parse_main_sheet(all_db_workbooks)
filenames_sheet = parse_filename_sheet(all_db_workbooks)
keywords_sheet = parse_keyword_sheet(all_db_workbooks)
advisors_sheet = parse_advisors_sheet(all_db_workbooks)

In [ ]:
catalog_sheet = parse_catalog_sheet()

In [ ]:
build_csv(main_sheet, catalog_sheet, filenames_sheet, keywords_sheet, advisors_sheet)


In [ ]:
# overlooked degrees are ones that aren't mapped legacy dept to current dept.  The legacy dept name is
# therefore transfering over to the current dept name.

# print(overlooked_degrees)

In [ ]:
# items in main sheet but not in catalog

# main_sheet_urns = {i for i in main_sheet}
# catalog_sheet_urns = {i for i in catalog_sheet}
# in_main_not_catalog = main_sheet_urns - catalog_sheet_urns

# print(len(in_main_not_catalog))

In [ ]:
# for urn, a_list in filenames_sheet.items():
#     for nt in a_list:
#         if nt.availability not in ('available', 'unrestricted'):
#             print(urn, nt.availability, nt.filename)

In [ ]:
# how often is adate a different year from pubDate?

count = 0
for urn, pack in main_sheet.items():
    if urn in catalog_sheet:
        if catalog_sheet[urn].PubDate:
            if remove_non_nums(catalog_sheet[urn].PubDate) != remove_non_nums(pack.adate)[:4]:
                count += 1
#                 print(urn, remove_non_nums(catalog_sheet[urn].PubDate), pack.rdate[:4])
        elif catalog_sheet[urn].SeriesDate:
            if remove_non_nums(catalog_sheet[urn].SeriesDate) != remove_non_nums(pack.adate)[:4]:
                count += 1
#                 print(urn, remove_non_nums(catalog_sheet[urn].SeriesDate), pack.rdate[:4])
print(count)

In [ ]:
listed_filenames = set()
duplicate_filenames = dict()

for urn, item_list in filenames_sheet.items():
    for item in item_list:
        if item.filename in listed_filenames:
            if item.filename in duplicate_filenames:
                duplicate_filenames[item.filename] += 1
            else:
                duplicate_filenames[item.filename] = 1
        else:
            listed_filenames.add(item.filename)

In [ ]:
len(duplicate_filenames)

In [ ]:
# displays which binaries share a name

# count = 0
# for n in range(12):
#     countdown = 13 - n
#     print('{} files with the name:'.format(countdown))
#     for k, v in duplicate_filenames.items():
#         if v == countdown:
#             print('\t', k)

In [ ]:
print(len(main_sheet))

In [ ]:
# which items have a weirdly formatted date

for urn in catalog_sheet:
    if not year_only.search(find_pub_date(catalog_sheet, urn)):
        print(urn, main_sheet[urn].sdate, main_sheet[urn].ddate, main_sheet[urn].rdate, main_sheet[urn].adate)
    if not year_month_day.search(find_defense_date(main_sheet, urn)):
        print(urn, main_sheet[urn].sdate, main_sheet[urn].ddate, main_sheet[urn].rdate, main_sheet[urn].adate)
    if not year_month_day.search(find_submission_date(main_sheet, urn)):
        print(urn, main_sheet[urn].sdate, main_sheet[urn].ddate, main_sheet[urn].rdate, main_sheet[urn].adate)

In [ ]:
items_extensions = dict()
for urn, pack in filenames_sheet.items():
    filetypes_found = dict()
    for bunch in pack:
        extension = bunch.filename.split('.')[-1].strip().lower()
        if extension in filetypes_found:
            filetypes_found[extension] += 1
        else:
            filetypes_found[extension] = 1
    items_extensions[urn] = filetypes_found

In [ ]:
# looking for items with supplementals
# makes a dict "has_supplementals" with key=urn, values=Tuple of FileNameNamedTuples

has_supplementals = dict()

for urn, list_of_filenamedtuples in filenames_sheet.items():
    if len(list_of_filenamedtuples) > 1:
        has_supplementals[urn] = list_of_filenamedtuples

    

In [ ]:
def get_all_urns_in(dtype):
    for urn, nt in main_sheet.items():
        if nt.dtype == dtype:
            yield urn

            
def do_pretty_print_of_supplementals_in(dtype):
    for urn in get_all_urns_in(dtype):
        if urn in has_supplementals:
            for i in has_supplementals[urn]:
                print(urn, i.filename, '\n', main_sheet[urn].title)
            print('\n')

In [ ]:
# do_pretty_print_of_supplementals_in('major_paper')

In [ ]:
# do_pretty_print_of_supplementals_in('thesis')

In [ ]:
# do_pretty_print_of_supplementals_in('dissertation')

In [ ]:
# copies etd binary folders that contain supplemental files into a new folder (for identification)
# it only works if the dest dirs do not exist (E.g., on first run)

# for urn, list_of_nts in has_supplementals.items():
#     source_dir = os.path.realpath('ETDbinaries/{}'.format(urn))
#     dest_dir = os.path.realpath('Has_Supplementals/{}'.format(urn))
#     shutil.copytree(source_dir, dest_dir)

In [ ]:
# has no pdf, or more than one pdf

# for urn, extensions_count in items_extensions.items():
#     if 'pdf' not in extensions_count:
#         print('{} has no pdf  {}'.format(urn, extensions_count))
#         break
#     elif extensions_count['pdf'] > 1:
#         print('{} has {} pdfs  {}'.format(urn, extensions_count['pdf'], extensions_count))
#         print(*('{} \t{}\n'.format(i.filename, i.size) for i in filenames_sheet[urn] if '.pdf' in i.filename.lower()))


In [ ]:
# read records scraped from DigitalCommons & compare against the records we expect (as listing in source files)

# disoutput = dict()

# with open('/home/francis/Desktop/DisOutput.csv', encoding='utf-8') as csvfile:
#     csvreader = csv.reader(csvfile, delimiter='\t')
#     for num, row in enumerate(csvreader):
#         if num == 0:
#             headers = (i for i in row)
#             DisOutput = namedtuple('DisOutput', headers)
#             continue
#         values = (i.lower().replace('...','') for i in row)
#         item = DisOutput(*values)
#         if item.keynumber not in test_items:
#             disoutput[item.keynumber] = item
        
        
# thesoutput = dict()

# with open('/home/francis/Desktop/ThesOutput.csv', encoding='utf-8') as csvfile:
#     csvreader = csv.reader(csvfile, delimiter='\t')
#     for num, row in enumerate(csvreader):
#         if num == 0:
#             headers = (i for i in row)
#             ThesOutput = namedtuple('ThesOutput', headers)
#             continue
#         values = (i.lower().replace('...','') for i in row)
#         item = ThesOutput(*values)
#         if item.keynumber not in test_items:
#             thesoutput[item.keynumber] = item
            
            
# all_disoutput_keynumber = set(i for i in disoutput)
# all_thesoutput_keynumber = set(i for i in thesoutput)

In [ ]:
# duplicate_disoutput = set()

# for keynumbera, disnta in disoutput.items():
#     for keynumberb, disntb in disoutput.items():
#         if keynumbera != keynumberb and disnta.title == disntb.title and disnta.author == disntb.author:
#             duplicate_disoutput.add((keynumbera, keynumberb))

In [ ]:
# duplicate_disoutput

In [ ]:
# for urn, mainsnt in main_sheet.items():
#     if 'BERRINGER' in mainsnt.last_name.upper():
#         print(urn, mainsnt.title)

In [ ]:
# duplicate_thesoutput = set()

# for keynumbera, thesnta in thesoutput.items():
#     for keynumberb, thesntb in thesoutput.items():
#         if keynumbera != keynumberb and thesnta.title == thesntb.title and thesnta.author == thesntb.author:
#             duplicate_thesoutput.add((keynumbera, keynumberb))

In [ ]:
# duplicate_thesoutput

In [ ]:
# for urn, mainsnt in main_sheet.items():
#     if 'YANN' in mainsnt.last_name.upper():
#         print(urn, mainsnt.title)

In [ ]:
# all_urns = {i for i in catalog_sheet}
# for i in main_sheet:
#     all_urns.add(i)

In [ ]:
# len(all_urns)

In [ ]:
for urn, nt in main_sheet.items():
    print(type(nt.rdate))
    print(nt.rdate)
    break

In [ ]:
record = main_sheet['etd-11032004-034832']

In [ ]:
record.title

In [ ]:
record = catalog_sheet['etd-11032004-034832']

In [ ]:
record.Title

In [ ]:
record.Subtitle